In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

Using TensorFlow backend.


In [2]:
def createOneHot(inFile,numberOfClasses):
    outFile = np.zeros((inFile.shape[0],inFile.shape[1], numberOfClasses));
    for i in range(0,inFile.shape[0]):
        for j in range(0,inFile.shape[1]):
            outFile[i,j,inFile[i,j]] = 1;
    return outFile;

In [29]:
imagePath1 = '/Volumes/mac_jannic_2017/thanujan/Datasets/Ximea_Tamron/20170622/Orthomosaics/Band';
imagePath2 = '/Volumes/mac_jannic_2017/thanujan/Datasets/Ximea_Tamron/20170622/VIS_Orthomosaics/Band';
amountOfClasses = 9;
inputDataShape = (1082,1406,41);

model = Sequential([
    Conv2D(64, 5, input_shape=(1082,1406,41), activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 5, activation='relu'),
    MaxPooling2D(),
    Conv2DTranspose(128, 5, strides=2, activation='relu'),
    Conv2DTranspose(64, 5, strides=2, activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])

model.compile(optimizer='adagrad',
              loss='categorical_crossentropy',
              metrics=['accuracy']);

In [10]:
# Turn 41 orthomosaics into one array. 
inputData = np.zeros(inputDataShape)
for i in range(1,25):
    temp= imageio.imread(imagePath1 + str(i) + '.png');
    inputData[:,:,i] = temp[:,:,0];
for i in range(1,16):
    temp= imageio.imread(imagePath2 + str(i) + '.png');
    inputData[:,:,i+25] = temp[:,:,0];
np.save('trainingData',inputData)

In [37]:
trainDataIn = np.load('trainingData.npy');
trainData = np.expand_dims(trainDataIn,axis=0)

temp = sio.loadmat('trainingLabels.mat');
trainLabelsIn = temp['labeledPicture'];
trainOneHotLabels = createOneHot(trainLabelsIn, 9);
trainOneHotLabelsDim0 = np.expand_dims(trainOneHotLabels,axis=0)

In [40]:

model.fit(trainData, trainOneHotLabelsDim0, epochs=10);


ValueError: Error when checking target: expected conv2d_9 to have shape (1077, 1401, 9) but got array with shape (1082, 1406, 9)

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 1078, 1402, 64)    65664     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 539, 701, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 535, 697, 128)     204928    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 267, 348, 128)     0         
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 537, 699, 128)     409728    
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 1077, 1401, 64)    204864    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 1077, 1401, 9)     585       
Total para

In [39]:
#np.reshape(train_data,(,1082,1406,41));
test = np.expand_dims(trainDataIn,axis=0)
print(trainDataIn.shape)
print(trainOneHotLabelsDim0.shape)
print(test.shape)

(1082, 1406, 41)
(1, 1082, 1406, 9)
(1, 1082, 1406, 41)
